In [1]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time
from typing import Tuple

import xgboost as xgb
from sklearn import metrics

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric
#from messaging import send_message

pd.set_option('display.max_columns', None)

In [2]:
# metrics in xgboost format

def metric_recall_at4(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    y_true = dtrain.get_label()
    return 'recall_at4', compute_recall_at4(y_true, predt)

def metric_normalized_gini(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    y_true = dtrain.get_label()
    return 'norm_gini', compute_normalized_gini(y_true, predt)

def metric_amex(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    y_true = dtrain.get_label()
    return 'amex_metric', compute_amex_metric(y_true, predt)

In [3]:
# CONFIG PARAMS
N_REPEATS = 5
DATASET_VERSION = "06"

In [4]:
OOF_PATH = Path(f"../data/oof/xgboost-dsv{DATASET_VERSION}")
SUB_PATH = Path(f"../data/subs/xgboost-dsv{DATASET_VERSION}")
ART_PATH = Path(f"../artifacts/xgboost-dsv{DATASET_VERSION}")

if not OOF_PATH.exists():
    OOF_PATH.mkdir(parents=True, exist_ok=True)
if not SUB_PATH.exists():
    SUB_PATH.mkdir(parents=True, exist_ok=True)
if not ART_PATH.exists():
    ART_PATH.mkdir(parents=True, exist_ok=True)

***
## load and prepare data

In [5]:
train = pd.read_parquet(f"../data/processed/dsv{DATASET_VERSION}/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [6]:
input_feats = train.columns.tolist()
len(input_feats)

1308

In [7]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
del train_labels
gc.collect()

517

***
## model training

train with repeated cross validation

In [8]:
model_params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'seed': 2112,
    'disable_default_eval_metric':True,
    # general params
    'eta': 0.05,
    'max_depth': 5,
    'tree_method': 'hist',
    'grow_policy': 'depthwise',
    'colsample_bytree': 0.1,
    'subsample': 0.8,
    'iterations': 4400,
    # regularization
    'min_child_weight': 100,
    'alpha': 28.291386794681554,
    'lambda': 3.5708647839613525,
    'max_delta_step': 0.7209862295680118,    
    'min_split_loss': 1.4540866337770056e-05,
    'max_bin': 1023,
}
MAX_ITERATIONS = model_params.pop("iterations")
print(MAX_ITERATIONS)

4400


In [9]:
def train_models(dataframe: pd.DataFrame, n_folds: int = 5,) -> tuple:
    
    models = list()
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for fold in range(n_folds):
        
        seed = repetition*5 + fold
        _model_params = dict(model_params)
        _model_params["seed"] = seed
        
        print(f" training model {fold+1}/{n_folds} ".center(100, "#"))
        print("seed:", seed)
        
        train_df = dataframe.query("fold != @fold").copy()
        valid_df = dataframe.query("fold == @fold").copy()
                                
        train_dset = xgb.DMatrix(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
        )
        valid_dset = xgb.DMatrix(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
        )
        model = xgb.train(
            params = _model_params,
            dtrain=train_dset,
            num_boost_round=MAX_ITERATIONS,
            evals=[(valid_dset,"eval"), ],
            custom_metric=metric_amex,
            verbose_eval=50,
        )
          
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset)
        
        models.append(model)
        del train_df,valid_df,train_dset,valid_dset
        gc.collect()
    
    return models,oof

In [10]:
# implement repeated cross validation
sorted(glob("../data/processed/cv*.csv"))

['../data/processed/cv0.csv',
 '../data/processed/cv1.csv',
 '../data/processed/cv2.csv',
 '../data/processed/cv3.csv',
 '../data/processed/cv4.csv',
 '../data/processed/cv5.csv',
 '../data/processed/cv6.csv',
 '../data/processed/cv7.csv',
 '../data/processed/cv8.csv',
 '../data/processed/cv9.csv']

In [11]:
%%time 

all_models = list()
all_oof_dfs = list()

for repetition in range(N_REPEATS):
    print(f" repeated cross-validation step: {repetition+1}/{N_REPEATS} ".center(100, "#"))

    folds = pd.read_csv(f'../data/processed/cv{repetition}.csv', index_col="customer_ID")
    _train = pd.merge(train, folds, how="inner", left_index=True, right_index=True).reset_index(drop=True)
    
    tic = time.time()
    models,oof = train_models(_train, n_folds=5)
    tac = time.time()
    print(f"Training time: {(tac-tic)/60} min.")
          
    # oof metrics
    print("OOF recall_at4:", compute_recall_at4(oof.target.values, oof.pred.values))
    print("OOF normalized_gini:", compute_normalized_gini(oof.target.values, oof.pred.values))
    print("OOF competition metric:", compute_amex_metric(oof.target.values, oof.pred.values))
    
    all_models.append(models)
    all_oof_dfs.append(oof)
    
    # save oof predictions
    oof.to_csv(OOF_PATH/f"oof-cv{repetition}.csv", index=False)
    
    # save models
    #for fold,_model in enumerate(models):
    #    _model.save_model(ART_PATH/f"/model-cv{repetition}-fold{fold}.cbm", format="cbm")
          
    del _train, folds; gc.collect()

############################### repeated cross-validation step: 1/5 ################################


######################################## training model 1/5 ########################################
seed: 0


[0]	eval-amex_metric:0.56370


[50]	eval-amex_metric:0.75632


[100]	eval-amex_metric:0.77060


...

[4300]	eval-amex_metric:0.79399


[4350]	eval-amex_metric:0.79388


[4399]	eval-amex_metric:0.79366


Training time: 252.83386897643408 min.
OOF recall_at4: 0.6669724307402296
OOF normalized_gini: 0.9252459832766
OOF competition metric: 0.7961092070084148


CPU times: user 20d 9h 53min 25s, sys: 5min 29s, total: 20d 9h 58min 54s
Wall time: 21h 42min 28s


In [12]:
results = list()

for oof in all_oof_dfs:  
    r = {
        "recall_at4": compute_recall_at4(oof.target.values, oof.pred.values),
        "gini": compute_normalized_gini(oof.target.values, oof.pred.values),
        "metric": compute_amex_metric(oof.target.values, oof.pred.values),
    }
    results.append(r)

results = pd.DataFrame(results)
display(results)

print("\nmean:")
display(results.mean(axis=0))

print("\nstd:")
display(results.std(axis=0))

,recall_at4,gini,metric
0,0.667839,0.925119,0.796479
1,0.667503,0.925132,0.796317
2,0.666644,0.925235,0.795940
3,0.666821,0.925113,0.795967
4,0.666972,0.925246,0.796109



mean:


recall_at4    0.667156
gini          0.925169
metric        0.796162
dtype: float64


std:


recall_at4    0.000499
gini          0.000066
metric        0.000232
dtype: float64

In [13]:
del train
gc.collect()

21

***
## make predictions and submit

In [14]:
def make_predictions(
        dataframe:pd.DataFrame, 
        input_feats:list,
        models:list,
    ) -> np.array:
    preds = list()
    for model in models:
        _dataframe = dataframe.copy()
        _dataframe_casted = xgb.DMatrix(data=_dataframe[input_feats])
        preds.append( model.predict(_dataframe_casted))
    return np.mean(preds, axis=0)   

In [15]:
test = pd.read_parquet(f"../data/processed/dsv{DATASET_VERSION}/test.parquet")
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [16]:
%%time

all_preds = list()

for repetition in range(N_REPEATS):
    if "prediction" in sub.columns:
        sub.drop("prediction", axis=1, inplace=True)
    if "prediction" in test.columns:
        test.drop("prediction", axis=1, inplace=True)
        
    models = all_models[repetition]
    preds = make_predictions(test, input_feats, models)
    all_preds.append(preds)
       
    test["prediction"] = preds
    sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
    assert sub.prediction.isna().sum() == 0
    sub.to_csv(SUB_PATH/f"submission-cv{repetition}.csv", index=False)

CPU times: user 1h 40min 19s, sys: 2min 35s, total: 1h 42min 55s
Wall time: 8min 30s


In [17]:
%%time
# predict using all the trained models
if "prediction" in sub.columns:
    sub.drop("prediction", axis=1, inplace=True)
if "prediction" in test.columns:
    test.drop("prediction", axis=1, inplace=True)

test["prediction"] = np.mean(all_preds, axis=0)
sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
assert sub.prediction.isna().sum() == 0
sub.to_csv(SUB_PATH/f"submission-all.csv", index=False)

CPU times: user 4.33 s, sys: 2.22 s, total: 6.55 s
Wall time: 6.85 s


***